# 01. Prepare input Data 
## split original ECG signal into frames, saving it scaled.

In [1]:
val scale = 1/200.0
val samplesPerSecond = 360

val WINDOW = 120   // samples per window
val WindowsPerFrame = 30
val STEP = 2
val SAMPLES = 200000

//val inputDataFile = "a02.dat"

val inputDataFile = "105s1.dat"


val samplesPerFrame = WindowsPerFrame * WINDOW
val secondPerFrame = samplesPerFrame / samplesPerSecond

scale = 0.005
samplesPerSecond = 360
WINDOW = 120
WindowsPerFrame = 30
STEP = 2
SAMPLES = 200000
inputDataFile = 105s1.dat
samplesPerFrame = 3600
secondPerFrame = 10


10

In [2]:
import java.io._                                                        
import scala.collection.mutable.ListBuffer   //bE CAREFULL With java imports, better before scala.

In [7]:
// Databricks

//dbutils.fs.mkdirs("/FileStore/fxo/ted")
//dbutils.fs.ls("FileStore")

//val rootDir = "FileStore/fxo/ted/"
//val fileName = "/dbfs/" + rootDir +"a02.dat"

// GCP

val rootDir = "/home/fsainz/data/"
val hdfsOutPath = "/user/fsainz/data/out/"
val fileName = rootDir + inputDataFile 

val framesFileName = "frames-"+ inputDataFile.split('.')(0)+"_" + secondPerFrame +"sLE.csv"


rootDir = /home/fsainz/data/
hdfsOutPath = /user/fsainz/data/out/
fileName = /home/fsainz/data/105s1.dat
framesFileName = frames-105s1_10sLE.csv


lastException: Throwable = null


frames-105s1_10sLE.csv

In [8]:
println("Generating frames of " + samplesPerFrame + " samples (" + 
        + secondPerFrame + "s.) long. ")
        println("OUTPUTFILE in hdfs  =" + hdfsOutPath+framesFileName)

Generating frames of 3600 samples (10s.) long. 
OUTPUTFILE in hdfs  =/user/fsainz/data/out/frames-105s1_10sLE.csv


In [9]:


//val rawData = sc.textFile("/FileStore/tables/gqh5mjl51507524821196/a02.dat", numPartitions)
var inFile =  new File(fileName)
var InFileStream = new FileInputStream( inFile )
val input : DataInputStream  = new DataInputStream(InFileStream)

var frames = scala.collection.mutable.ArrayBuffer.empty[Array[Double]]

var pos = 0
var available = 0


val frameBytes =  samplesPerFrame * 2 // read shorts 2 bytes.
val nFrames = inFile.length / frameBytes
println("Input file length =" + inFile.length  )
println("nFrames = " + nFrames)

var dataFrame = Array[Double]()
for (  w  <- 1 to nFrames.toInt ) {
    
   dataFrame = new Array[Double](samplesPerFrame)
   for ( i <- 0 until samplesPerFrame) {
     // read little indiean and scale
    val value = java.lang.Short.reverseBytes(input.readShort() )  // two bytes
    dataFrame( i ) = (value.toDouble * scale) 
    
  }
  frames.append(dataFrame)
   
}
input.close()

Input file length =1300000
nFrames = 180


inFile = /home/fsainz/data/105s1.dat
InFileStream = java.io.FileInputStream@3591fdbe
input = java.io.DataInputStream@1dc74e06
frames = ArrayBuffer(Array(4.675, 4.675, 4.675, 4.675, 4.675, 4.675, 4.675, 4.675, 4.67, 4.69, 4.6850000000000005, 4.715, 4.735, 4.755, 4.765, 4.78, 4.795, 4.815, 4.835, 4.86, 4.875, 4.8950000000000005, 4.88, 4.8950000000000005, 4.91, 4.925, 4.93, 4.925, 4.915, 4.915, 4.92, 4.915, 4.91, 4.905, 4.885, 4.885, 4.9, 4.885, 4.88, 4.865, 4.84, 4.845, 4.855, 4.8500000000000005, 4.8500000000000005, 4.835, 4.815, 4.815, 4.825, 4.83, 4.82, 4.815, 4.805, 4.8100000000000005, 4.805, 4.825, 4.82, 4.8100000000000005, 4.805, 4.805, 4.8, 4.805, 4.815, 4.81000000000...


ArrayBuffer([D@4ec2d06a, [D@576f215f, [D@b6e073c, [D@401fe664, [D@18863f8d, [D@364d801b, [D@4d612609, [D@7272d507, [D@5b21e693, [D@1b6e8b4, [D@4523d7e5, [D@219df8ab, [D@755c8775, [D@43193222, [D@185e495b, [D@5648b25b, [D@47bef0c8, [D@4d3c423f, [D@27c2de52, [D@340b6a9b, [D@403e462, [D@2b017367, [D@1a00081b, [D@7530e672, [D@6e0b3c42, [D@4eb1e0af, [D@54eb79b2, [D@261a062b, [D@11c4d786, [D@50b1e989, [D@5f982cc4, [D@29c55242, [D@1e0ac81b, [D@412887f2, [D@5f487d1f, [D@5c1c73f6, [D@149e327e, [D@118c3008, [D@db017a3, [D@320c4620, [D@57e49fdd, [D@2798e959, [D@2a98ef06, [D@7a759133, [D@91ce3d7, [D@692e018c, [D@4233370, [D@5289d6ce, [D@3ce59513, [D@25dec162, [D@38fbde1a, [D@2458406d, [D@348514f8, [D@670316db, [D@1ee1f2f, [D@67d37c57, [D@7bed9681, [D@427f2e9, [D@75c19225, [D@98c690e, [D@1e42af20, [D@2549b6bb, [D@5c60588d, [D@36ce8e16, [D@3d7e1c40, [D@55ea993a, [D@b10173a, [D@49ec8670, [D@43d1f11, [D@59f65e5, [D@e3d0bab, [D@234fd31a, [D@4fe695f5, [D@31a871a1, [D@572e446c, [D@a248a90, [D@7196ff1c, [

In [15]:
val csvFilePath = hdfsOutPath + framesFileName
// FileWriter

//val file = new File(csvFilePath)
//val bw = new BufferedWriter(new FileWriter(file))
import org.apache.hadoop.fs.Path;
val conf = sc.hadoopConfiguration
val fs = org.apache.hadoop.fs.FileSystem.get(conf)

val output = fs.create(new Path(csvFilePath))


var line = ""
for ( f <- frames) {
  f.foreach( x => line +=  x.toString + "," )
  output.write((line.substring(0,line.length-1) + "\n").getBytes())
  line = ""
 
}
output.close()
println(csvFilePath + " Written")

Waiting for a Spark session to start...

/user/fsainz/data/out/frames-105s1_10sLE.csv Written


csvFilePath = /user/fsainz/data/out/frames-105s1_10sLE.csv
conf = Configuration: core-default.xml, core-site.xml, mapred-default.xml, mapred-site.xml, yarn-default.xml, yarn-site.xml, hdfs-default.xml, hdfs-site.xml, file:/etc/hive/conf.dist/hive-site.xml
fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_-494003913_83, ugi=root (auth:SIMPLE)]]
output = org.apache.hadoop.hdfs.client.HdfsDataOutputStream@3830bb88


line: String = ""


org.apache.hadoop.hdfs.client.HdfsDataOutputStream@3830bb88